In [1]:
import numpy as np 
import pandas as pd 
import os
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from tqdm.autonotebook import tqdm

plt.style.use('dark_background')
warnings.filterwarnings('ignore', 'FutureWarning')

/tmp/ipykernel_27/847551157.py:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Create Dataset

In [2]:
class Dataset2Class(Dataset):
    def __init__(self, path_dir1: str, path_dir2: str):
        super().__init__()
        
        self.path_dir1 = path_dir1
        self.path_dir2 = path_dir2
        
        # Получение списков, содержащих имена файлов и директорий в 
        # каталогах path_dir1, path_dir2
        self.dir1_list = sorted(os.listdir(path_dir1))
        self.dir2_list = sorted(os.listdir(path_dir2))
        
        self.len_dir1_list = len(self.dir1_list)
    
    def __len__(self):
        return len(self.dir1_list) + len(self.dir2_list)
    
    def __getitem__(self, idx):
        if idx < self.len_dir1_list:
            class_id = 0
            img_path = os.path.join(self.path_dir1, self.dir1_list[idx])
        else:
            class_id = 1
            idx -= self.len_dir1_list
            img_path = os.path.join(self.path_dir2, self.dir2_list[idx])
            
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img / 255.0
        img = cv2.resize(img, (50, 50), interpolation=cv2.INTER_AREA)
        img = img.transpose((2, 0, 1))
        t_img = torch.from_numpy(img)
        t_class_id = torch.tensor(class_id)
        
        return {'img': t_img, 'label': t_class_id}

In [3]:
# 0 - dogs, 1 - cats

train_dogs_path = '/kaggle/input/dog-vs-cat-fastai/dogscats/train/dogs'
train_cats_path = '/kaggle/input/dog-vs-cat-fastai/dogscats/train/cats'

test_dogs_path = '/kaggle/input/dog-vs-cat-fastai/dogscats/valid/dogs'
test_cats_path = '/kaggle/input/dog-vs-cat-fastai/dogscats/valid/cats'

train_dataset = Dataset2Class(train_dogs_path, train_cats_path)
test_dataset = Dataset2Class(test_dogs_path, test_cats_path)

## Create Dataloader

In [4]:
batch_size = 16
train_loader = DataLoader(train_dataset, 
                          shuffle=True, 
                          drop_last=True,
                          batch_size=batch_size,
                         )
test_loader = DataLoader(test_dataset,
                        shuffle=True,
                        batch_size=batch_size,
                        )

## Architecture CNN

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, stride=1)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1)
        self.conv3 = nn.Conv2d(64, 64, 3, stride=1)
        self.conv4 = nn.Conv2d(64, 64, 3, stride=1)
        
        self.maxpool = nn.MaxPool2d(2, 2)
        self.adaptivepool = nn.AdaptiveAvgPool2d((1, 1)) # Автоматически рассчитывает значения
                                                        # для достижения указанного рамера
        self.leaky_relu = nn.LeakyReLU(0.2)
        
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(64, 10)
        self.linear2 = nn.Linear(10, 2)
        
    def forward(self, x):
        # Сверточная сеть
        x = self.conv1(x)
        x = self.leaky_relu(x)
        x = self.maxpool(x)

        x = self.conv2(x)
        x = self.leaky_relu(x)
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = self.leaky_relu(x)
        x = self.adaptivepool(x)
        
        # Полносвязная сеть
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.leaky_relu(x)
        x = self.linear2(x)
        
        return x

In [6]:
model = ConvNet()

In [7]:
print(model)

ConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (adaptivepool): AdaptiveAvgPool2d(output_size=(1, 1))
  (leaky_relu): LeakyReLU(negative_slope=0.2)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=64, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=2, bias=True)
)


In [8]:
for sample in train_loader:
    img = sample['img']
    model(img)
    break

## Learn CNN

In [9]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

model = model.to(DEVICE)
EPOCHS = 10

In [10]:
def accuracy(pred: torch.Tensor, target: torch.Tensor) -> int:
    proba = F.softmax(pred, dim=1)
    true = (torch.argmax(pred.detach(), dim=1) == torch.argmax(target, dim=1)).sum()
    return true.to(torch.float32).mean().detach().cpu().numpy().item()

In [11]:
train_loss = []
total_loss = []

val_loss = []
total_val_loss = []

acc_train = []
acc_val = []

size = len(train_dataset)

In [12]:
for epoch in tqdm(range(EPOCHS)):
    print(f'Epochs {epoch} \n------------------------')
    for iteration, tensor in enumerate(train_loader):
        
        X_batch = tensor.get('img').to(DEVICE)
        y_batch = tensor.get('label').to(DEVICE)
        y_batch = F.one_hot(y_batch, 2).to(torch.float32)
        
        model.train()
        optimizer.zero_grad()
        
        outputs = model(X_batch)
        loss = loss_function(outputs, y_batch)
    
        loss.backward()
        
        acc = accuracy(outputs, y_batch)
        
        acc_train.append(acc)
        train_loss.append(loss.detach().cpu().numpy().item())
        
        optimizer.step()
        
        current = (iteration + 1) * len(X_batch)
        if current % 1000 == 0:
            print(f'loss: {loss:>7f} [{current}|{size:>2d}] Accuracy: {np.mean(acc_train)}')
    
    total_loss.append(np.mean(train_loss)) # для графика
    for iteration, tensor in enumerate(test_loader):
        X_val = tensor.get('img').to(DEVICE)
        y_val = tensor.get('label').to(DEVICE)
        y_val = F.one_hot(y_val, 2).to(torch.float32)
        with torch.no_grad():
            model.eval()
            
            outputs = model(X_val)
            
            loss = loss_function(outputs, y_val)
            acc = accuracy(outputs, y_val)
            
            acc_val.append(acc)
            val_loss.append(loss.detach().cpu().numpy().item())
            
    total_val_loss.append(np.mean(val_loss)) # для графика
    print(f'Validation Error:\nAccuracy: {np.mean(acc_val)} Avg Loss: {np.mean(val_loss)}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epochs 0 
------------------------
loss: 0.687894 [2000|23000] Accuracy: 8.248
loss: 0.685286 [4000|23000] Accuracy: 8.2
loss: 0.693420 [6000|23000] Accuracy: 8.304
loss: 0.678937 [8000|23000] Accuracy: 8.382
loss: 0.680900 [10000|23000] Accuracy: 8.496
loss: 0.677943 [12000|23000] Accuracy: 8.586666666666666
loss: 0.649975 [14000|23000] Accuracy: 8.658285714285714
loss: 0.580931 [16000|23000] Accuracy: 8.786
loss: 0.636467 [18000|23000] Accuracy: 8.862222222222222
loss: 0.554196 [20000|23000] Accuracy: 8.9712
loss: 0.617845 [22000|23000] Accuracy: 9.03490909090909
Validation Error:
Accuracy: 9.112 Avg Loss: 0.6773789455890655
Epochs 1 
------------------------
loss: 0.654565 [2000|23000] Accuracy: 9.139564660691422
loss: 0.669052 [4000|23000] Accuracy: 9.200948429164196
loss: 0.702392 [6000|23000] Accuracy: 9.277041942604857
loss: 0.636465 [8000|23000] Accuracy: 9.364997418688693
loss: 0.657438 [10000|23000] Accuracy: 9.405916585838991
loss: 0.791971 [12000|23000] Accuracy: 9.46822130